In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('player_stats_initial.csv', delimiter=';')

In [3]:
df.isnull().sum()

id          0
Player      0
Nation      1
Position    0
Squad       0
           ..
OG          0
Recov       0
AerWon      0
AerLost     0
AerWon%     0
Length: 124, dtype: int64

In [4]:
df.isna().sum()

id          0
Player      0
Nation      1
Position    0
Squad       0
           ..
OG          0
Recov       0
AerWon      0
AerLost     0
AerWon%     0
Length: 124, dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.rename(columns={'Assists': 'Assists/90'}, inplace=True)


In [7]:
print('Assists' in df.columns)
print('Assists/90' in df.columns)

False
True


In [8]:
df['Assists'] = (df['Assists/90'] * (df['Min'] / 90)).round().astype(int)
df['Goals/90'] = (df['Goals'] / df['Min']) * 90
df['Goals/90'] = df['Goals/90'].round(2)


In [9]:
print(df[['Player', 'Assists/90', 'Min', 'Assists']].head())

              Player  Assists/90   Min  Assists
0   Brenden Aaronson        0.11  1596        2
1   Yunis Abdelhamid        0.05  1980        1
2      Himad Abdelli        0.00   770        0
3  Salis Abdul Samed        0.00  1799        0
4    Laurent Abergel        0.08  1165        1


In [10]:
variables = ['Shots', 'SoT', 'Int', 'TklWon', 'Recov', 'Fls', 'CrdY', 'CrdR']

In [11]:
df.rename(columns={var: var + '/90' for var in variables}, inplace=True)

In [12]:
for var in variables:
    print(var in df.columns)

False
False
False
False
False
False
False
False


In [13]:
for var in variables:
    df[var] = (df[var + '/90'] * (df['Min'] / 90)).round().astype(int)

In [14]:
variables_per90 = [var + '/90' for var in variables]
print(df[['Player'] + ['Min'] + variables + variables_per90].head())

              Player   Min  Shots  SoT  Int  TklWon  Recov  Fls  CrdY  CrdR  \
0   Brenden Aaronson  1596     27    5    1       9     86   11     2     0   
1   Yunis Abdelhamid  1980     19    1   44      35    146   29     2     0   
2      Himad Abdelli   770      9    3    8      12     70   15     1     0   
3  Salis Abdul Samed  1799     12    3   22      16    132   49     3     0   
4    Laurent Abergel  1165      4    0   15      26     84   12     2     0   

   Shots/90  SoT/90  Int/90  TklWon/90  Recov/90  Fls/90  CrdY/90  CrdR/90  
0      1.53    0.28    0.06       0.51      4.86    0.62     0.11      0.0  
1      0.86    0.05    2.00       1.59      6.64    1.32     0.09      0.0  
2      1.05    0.35    0.93       1.40      8.14    1.74     0.12      0.0  
3      0.60    0.15    1.10       0.80      6.60    2.45     0.15      0.0  
4      0.31    0.00    1.16       2.02      6.51    0.93     0.16      0.0  


In [15]:
df['Position'].unique()

array(['MFFW', 'DF', 'MF', 'FWMF', 'FW', 'DFFW', 'MFDF', 'GK', 'DFMF',
       'FWDF'], dtype=object)

In [16]:
df['Position'] = df['Position'].apply(lambda x: x[:2] + ',' + x[2:] if len(x) > 2 else x)

In [17]:
max_games = df.groupby('Squad')['MP'].max().reset_index()
max_games.rename(columns={'MP': 'MaxGames'}, inplace=True)


In [18]:
df = pd.merge(df, max_games, on='Squad')

In [19]:
df.head()

,id,Player,Nation,Position,Squad,Comp,Age,Born,MP,Starts,...,Goals/90,Shots,SoT,Int,TklWon,Recov,Fls,CrdY,CrdR,MaxGames
0,1,Brenden Aaronson,USA,"MF,FW",Leeds United,Premier League,22,2000,20,19,...,0.06,27,5,1,9,86,11,2,0,21
1,2,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,35,1987,22,22,...,0.00,19,1,44,35,146,29,2,0,22
2,3,Himad Abdelli,FRA,"MF,FW",Angers,Ligue 1,23,1999,14,8,...,0.00,9,3,8,12,70,15,1,0,22
3,4,Salis Abdul Samed,GHA,MF,Lens,Ligue 1,22,2000,20,20,...,0.05,12,3,22,16,132,49,3,0,22
4,5,Laurent Abergel,FRA,MF,Lorient,Ligue 1,30,1993,15,15,...,0.00,4,0,15,26,84,12,2,0,22


In [20]:
df['MaxPossibleMinutes'] = df['MaxGames'] * 90
df['MinutesOnBench'] = df['MaxPossibleMinutes'] - df['Min']
df['PlayBySuplente'] = df['MP'] - df['Starts']
df['NotPlayed'] = df['MaxGames'] - df['MP']

In [21]:
df.to_csv('player_stats.csv', index=False)